## Creating a variable clustering algorithm that is similar to SAS varclus
1. PCA-based recursive decomposition
2. stopping critiera
3. plotting
4. OOP

In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.decomposition import PCA
from sklearn.preprocessing import scale

from decomposition.var_clus import VarClus, Cluster

In [2]:
# Fake data
feature_df, label_df = make_classification(n_samples=int(1e4), n_features=50)
feature_df = scale(feature_df)

In [3]:
# Make sure data is zscored
np.max(np.abs(feature_df.mean(axis=0))), np.max((feature_df.std()))

(1.1038947533847931e-16, 1.0)

In [4]:
feature_df = pd.DataFrame(feature_df, columns=['feature_' + str(i) for i in range(50)])

In [5]:
# Some simple tests
test_clus = Cluster(feature_df)

In [9]:
test_clus.run_pca()
type(test_clus.pca_corr), type(test_clus.pca_features)

(list, list)

In [12]:
test_clus.return_all_leaves() == test_clus

True